In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("./csv/AMZN.csv")
df.head()

In [ ]:
price_df = df.loc[:, ["Date", "Adj Close"]].copy()
price_df.head()

In [ ]:
## Date 컬럼 인덱스 변환
price_df.set_index(["Date"], inplace=True)
price_df.head()

In [ ]:
## 이동 평균선 구하기 -> 데이터양 20개의 평균을 구해서 컬럼에 대입
price_df["center"] = price_df["Adj Close"].rolling(20).mean()
price_df.head(25)

In [ ]:
# 상단 밴드 생성 ( 이동 평균선 + 2 *표준편차 )
price_df["ub"] = price_df['center'] + 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

In [ ]:
# 하단 밴드 생성 ( 이동 평균선 - 2 * 표준편차 )
price_df["lb"] = price_df['center'] - 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

In [10]:
## 이동 평균선과 상단 밴드, 하단 밴드 구하는 함수를 생성
## 함수에 매개변수는 dataframe이 들어가는 매개변수 1개, 
## 평균선을 그릴때 데이터의 갯수가 담기는 매개변수 1개

def bollinger(data, n):
    df = data.loc[:, ["Date", "Adj Close"]].copy()
    df.set_index(["Date"], inplace=True)
    df["center"] = df["Adj Close"].rolling(n).mean()
    df["ub"] = df["center"] + 2 * df["Adj Close"].rolling(n).std()
    df["lb"] = df["center"] - 2 * df["Adj Close"].rolling(n).std()
    return df

In [11]:
test = pd.read_csv("./csv/AAPL.csv")

In [ ]:
bollinger(test, 20)

In [14]:
bollinger_data = bollinger(df, 20)

In [15]:
# 기준일 지정
start_time = "2008-01-02"

In [ ]:
sample = bollinger_data.loc[start_time:]
sample.head()

In [ ]:
## 거래 내역 데이터프레임 생성
book = sample[["Adj Close"]].copy()
book

In [ ]:
## 비어있는 컬럼 추가
book['trade'] = ""
book.head()

In [19]:
## 거래 내역 데이터프레임을 생성하는 함수 생성
## 매개변수는 1개 dataframe이 들어가는 변수 1개

def create_trade_book(data):
    book = data[["Adj Close"]].copy()
    book["trade"] = ""
    return book

In [ ]:
book = create_trade_book(sample)
book.head()

In [27]:
## 구매 내역 데이터를 삽입
## 조건 : 상단 밴드보다 종가가 높은 경우
##          구매 상태이면
##              매도  trade = ""
##          구매 상태가 아니면 
##              아무 행동도 하지 않는다. trade = "" 
##        하단 밴드보다 낮은 경우
##          구매 상태이면
##              구매 상태를 유지 trade = "buy"
##          구매 상태가 아니면
##              구매 trade = "buy"
##        상단 과 하단 밴드 사이에 종가가 존재하면
##          구매 상태이면
##              구매 유지 trade = "buy"
##          구매 상태가 아니면
##              아무 행동도 하지 않는다. trade = ""

def trade(data, book):          ## data에는 상단, 하단 밴드 수치가 있는 데이터프레임 삽입
                                ## book에는 거래 내역 데이터프레임 삽입
    for i in data.index:
        if data.loc[i, "Adj Close"] > data.loc[i, "ub"]:    ## 종가가 상단 밴드보다 높은 경우
            book.loc[i, "trade"] = ""
        elif data.loc[i, "lb"] > data.loc[i, "Adj Close"]:    ## 종가가 하단 밴드보다 작은 경우
            book.loc[i, "trade"] = "buy"
        elif data.loc[i, "ub"] >= data.loc[i, "Adj Close"] and data.loc[i, "Adj Close"] >= data.loc[i, "lb"]:  
            ## 상단 밴드와 하단 밴드 사이에 종가가 존재하는 경우
            if book.shift(1).loc[i, "trade"] == "buy":    ## 구매 상태인 경우
                book.loc[i, "trade"] = "buy"
            else:
                book.loc[i, "trade"] = ""

    return book
  

In [28]:
trade_book = trade(sample, book)
trade_book.tail()

,Adj Close,trade
Date,,
2019-06-18,1901.369995,buy
2019-06-19,1908.790039,buy
2019-06-20,1918.189941,buy
2019-06-21,1911.300049,buy
2019-06-24,1907.953857,buy


In [29]:
## 손익 계산
def returns(book):
    rtn = 1.0
    book["return"] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, "trade"] == "buy" and book.shift(1).loc[i, "trade"] == "":   ## 구매한 시기
            buy = book.loc[i, "Adj Close"]
            print("진입일 : ", i , "진입 가격 : ", buy)
        elif book.loc[i, "trade"] == "" and book.shift(1).loc[i, "trade"] == "buy": ## 매도한 시기
            sell = book.loc[i, "Adj Close"]
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, "return"] = rtn
            print("청산일 : ", i, "진입 가격 : ", buy, "청산 가격 : ", sell, " | return : ", round(rtn, 4))
        if book.loc[i, "trade"] == "":      ## buy, sell 초기화
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, "return"]
        acc_rtn = acc_rtn * rtn  ## 누적 수익률 계산
        book.loc[i, "acc_rtn"] = acc_rtn    ##누적 수익률 컬럼 생성
    print("누적 수익률 : ", round(acc_rtn, 4))
    return (round(acc_rtn, 4))   

In [30]:
returns(trade_book)

진입일 :  2008-01-11 진입 가격 :  81.08000200000001
청산일 :  2008-03-24 진입 가격 :  81.08000200000001 청산 가격 :  75.949997  | return :  0.9367
진입일 :  2008-06-27 진입 가격 :  74.660004
청산일 :  2008-07-24 진입 가격 :  74.660004 청산 가격 :  78.720001  | return :  1.0544
진입일 :  2008-09-17 진입 가격 :  71.540001
청산일 :  2008-12-08 진입 가격 :  71.540001 청산 가격 :  51.41  | return :  0.7186
진입일 :  2009-05-13 진입 가격 :  74.190002
청산일 :  2009-06-01 진입 가격 :  74.190002 청산 가격 :  83.050003  | return :  1.1194
진입일 :  2009-07-07 진입 가격 :  75.629997
청산일 :  2009-07-20 진입 가격 :  75.629997 청산 가격 :  88.230003  | return :  1.1666
진입일 :  2009-08-31 진입 가격 :  81.190002
청산일 :  2009-09-16 진입 가격 :  81.190002 청산 가격 :  90.699997  | return :  1.1171
진입일 :  2010-05-04 진입 가격 :  129.830002
청산일 :  2010-08-04 진입 가격 :  129.830002 청산 가격 :  127.58000200000001  | return :  0.9827
진입일 :  2010-11-16 진입 가격 :  157.779999
청산일 :  2010-11-24 진입 가격 :  157.779999 청산 가격 :  177.25  | return :  1.1234
진입일 :  2011-01-21 진입 가격 :  177.419998
청산일 :  2011-03-30 진입 가격 :  177.41999

3.2493

In [37]:
import bollinger as blg 
import imp
imp.reload(blg)

<module 'bollinger' from 'c:\\Users\\moons\\Documents\\GitHub\\ubion-4\\220707\\bollinger.py'>

In [32]:
import FinanceDataReader as fdr
df_finance = fdr.DataReader("090430", "2010-01-02", "2018-12-31")
df_finance.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2010-01-04,93424,94424,92423,94224,16684,0.008563
2010-01-05,93324,94224,91523,91524,9988,-0.028655
2010-01-06,91123,91423,86022,86022,41732,-0.060115
2010-01-07,86722,87722,84121,84122,30516,-0.022087
2010-01-08,84521,85421,82321,82821,18959,-0.015466


In [ ]:
f_sample = blg.bollinger(df_finance, 20)
f_sample

In [ ]:
f_book = blg.create_trade_book(f_sample)
f_book.head()

In [ ]:
f_book = blg.trade(f_sample, f_book)
f_book.tail()

In [43]:
blg.returns(f_book)

진입일 :  2010-02-10 00:00:00 진입 가격 :  76720
청산일 :  2010-03-12 00:00:00 진입 가격 :  76720 청산 가격 :  83121  | return :  1.0834
진입일 :  2010-04-05 00:00:00 진입 가격 :  78820
청산일 :  2010-05-07 00:00:00 진입 가격 :  78820 청산 가격 :  88023  | return :  1.1168
진입일 :  2010-07-07 00:00:00 진입 가격 :  94824
청산일 :  2010-08-05 00:00:00 진입 가격 :  94824 청산 가격 :  104827  | return :  1.1055
진입일 :  2010-10-08 00:00:00 진입 가격 :  103527
청산일 :  2010-11-03 00:00:00 진입 가격 :  103527 청산 가격 :  108928  | return :  1.0522
진입일 :  2011-02-08 00:00:00 진입 가격 :  98625
청산일 :  2011-04-15 00:00:00 진입 가격 :  98625 청산 가격 :  109028  | return :  1.1055
진입일 :  2011-08-03 00:00:00 진입 가격 :  114730
청산일 :  2011-10-05 00:00:00 진입 가격 :  114730 청산 가격 :  125032  | return :  1.0898
진입일 :  2011-11-11 00:00:00 진입 가격 :  112829
청산일 :  2012-02-23 00:00:00 진입 가격 :  112829 청산 가격 :  106327  | return :  0.9424
진입일 :  2012-04-10 00:00:00 진입 가격 :  108428
청산일 :  2012-08-21 00:00:00 진입 가격 :  108428 청산 가격 :  107028  | return :  0.9871
진입일 :  2013-01-10 00:00:00 진입 가격 :

0.9496